In [ ]:
import flexpolyline as fp
import geopandas as gpd
import logging
import pandas as pd
import psycopg2
import requests
from shapely.geometry import Polygon

In [ ]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')

    cur = conn.cursor()
    
    return conn, cur

In [ ]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [ ]:
def get_ia(coords):
    payload = {
        'apiKey': 'KDUbLze9c55M07_jhPSR750KYGk9UxfX42DItWJWV8Y',
        'origin': coords,
        'range[type]': 'time',
        'range[values]': '300',
        'transportMode': 'car'
    }

    req = requests.get('https://isoline.router.hereapi.com/v8/isolines', params = payload)
    response = req.json()

    for item in response['isolines']:
        polygon = fp.decode(item['polygons'][0]['outer'])
        coords = [[coord[1], coord[0]] for coord in polygon]

    gdf = gpd.GeoDataFrame(geometry = [Polygon(coords)], crs = 'EPSG:4326')

    return gdf

In [ ]:
def get_sectors(conn, cur, polygon):
    query = f" \
        SELECT SC.cod_ibge \
        FROM PUBLIC.setores_censitarios AS SC \
        WHERE ST_Intersects(SC.geom, \'SRID=4326;{polygon}\');"
    
    try:
        cur.execute(query)
        result = [row[0] for row in cur.fetchall()]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def get_data(conn, cur, sectors):
    sectors = ', '.join(map(str, [sector for sector in sectors]))
    
    query = f" \
        SELECT \
          	SC.id_municipio   AS ID_MUNICIPIO, \
            SC.cod_ibge       AS ID_SETOR_CENSITARIO, \
            PRM.\"00_04\"     AS PRM_00_04, \
            PRM.\"05_09\"     AS PRM_05_09, \
            PRM.\"10_14\"     AS PRM_10_14, \
            PRM.\"15_19\"     AS PRM_15_19, \
            PRM.\"20_24\"     AS PRM_20_24, \
            PRM.\"25_29\"     AS PRM_25_29, \
            PRM.\"30_34\"     AS PRM_30_34, \
            PRM.\"35_39\"     AS PRM_35_39, \
            PRM.\"40_44\"     AS PRM_40_44, \
            PRM.\"45_49\"     AS PRM_45_49, \
            PRM.\"50_54\"     AS PRM_50_54, \
            PRM.\"55_59\"     AS PRM_55_59, \
            PRM.\"60_64\"     AS PRM_60_64, \
            PRM.\"65_69\"     AS PRM_65_69, \
            PRM.\"70_74\"     AS PRM_70_74, \
            PRM.\"75_79\"     AS PRM_75_79, \
            PRM.\"80_84\"     AS PRM_80_84, \
            PRM.\"85_89\"     AS PRM_85_89, \
            PRM.\"90_94\"     AS PRM_90_94, \
            PRM.\"95_99\"     AS PRM_95_99, \
            PRM.\"100\"       AS PRM_100, \
            PRF.\"00_04\"     AS PRF_00_04, \
            PRF.\"05_09\"     AS PRF_05_09, \
            PRF.\"10_14\"     AS PRF_10_14, \
            PRF.\"15_19\"     AS PRF_15_19, \
            PRF.\"20_24\"     AS PRF_20_24, \
            PRF.\"25_29\"     AS PRF_25_29, \
            PRF.\"30_34\"     AS PRF_30_34, \
            PRF.\"35_39\"     AS PRF_35_39, \
            PRF.\"40_44\"     AS PRF_40_44, \
            PRF.\"45_49\"     AS PRF_45_49, \
            PRF.\"50_54\"     AS PRF_50_54, \
            PRF.\"55_59\"     AS PRF_55_59, \
            PRF.\"60_64\"     AS PRF_60_64, \
            PRF.\"65_69\"     AS PRF_65_69, \
            PRF.\"70_74\"     AS PRF_70_74, \
            PRF.\"75_79\"     AS PRF_75_79, \
            PRF.\"80_84\"     AS PRF_80_84, \
            PRF.\"85_89\"     AS PRF_85_89, \
            PRF.\"90_94\"     AS PRF_90_94, \
            PRF.\"95_99\"     AS PRF_95_99, \
            PRF.\"100\"       AS PRF_100, \
            PIA.\"masculina\" AS PIA_M, \
            PIA.\"feminina\"  AS PIA_F, \
            PIA.\"total\"     AS PIA_T, \
            RS.\"00_04\"      AS RS_00_04, \
            RS.\"05_09\"      AS RS_05_09, \
            RS.\"10_14\"      AS RS_10_14, \
            RS.\"15_19\"      AS RS_15_19, \
            RS.\"20_24\"      AS RS_20_24, \
            RS.\"25_29\"      AS RS_25_29, \
            RS.\"30_34\"      AS RS_30_34, \
            RS.\"35_39\"      AS RS_35_39, \
            RS.\"40_44\"      AS RS_40_44, \
            RS.\"45_49\"      AS RS_45_49, \
            RS.\"50_54\"      AS RS_50_54, \
            RS.\"55_59\"      AS RS_55_59, \
            RS.\"60_64\"      AS RS_60_64, \
            RS.\"65_69\"      AS RS_65_69, \
            RS.\"70_74\"      AS RS_70_74, \
            RS.\"75_79\"      AS RS_75_79, \
            RS.\"80_84\"      AS RS_80_84, \
            RS.\"85_89\"      AS RS_85_89, \
            RS.\"90_94\"      AS RS_90_94, \
            RS.\"95_99\"      AS RS_95_99, \
            RS.\"100\"        AS RS_100, \
            IE.\"masculino\"  AS IE_M, \
            IE.\"feminino\"   AS IE_F, \
            DR.\"classe_ab\"  AS DR_CLASSE_AB, \
            DR.\"classe_c\"   AS DR_CLASSE_C, \
            DR.\"classe_d\"   AS DR_CLASSE_D, \
            DR.\"classe_e\"   AS DR_CLASSE_E, \
            RR.\"classe_a\"   AS RR_CLASSE_A, \
            RR.\"classe_b\"   AS RR_CLASSE_B, \
            RR.\"classe_c\"   AS RR_CLASSE_C, \
            RR.\"classe_d\"   AS RR_CLASSE_D, \
            RR.\"classe_e\"   AS RR_CLASSE_E \
        FROM PUBLIC.setores_censitarios AS SC \
        JOIN PUBLIC.populacao_residente_mas AS PRM ON SC.cod_ibge = PRM.id_setor_censitario \
        JOIN PUBLIC.populacao_residente_fem AS PRF ON SC.cod_ibge = PRF.id_setor_censitario \
        JOIN PUBLIC.populacao_idade_ativa AS PIA ON SC.cod_ibge = PIA.id_setor_censitario \
        JOIN PUBLIC.razao_sexo AS RS ON SC.cod_ibge = RS.id_setor_censitario \
        JOIN PUBLIC.indice_envelhecimento AS IE ON SC.cod_ibge = IE.id_setor_censitario \
        JOIN PUBLIC.domicilio_renda AS DR ON SC.cod_ibge = DR.id_setor_censitario \
        JOIN PUBLIC.responsavel_renda AS RR ON SC.cod_ibge = RR.id_setor_censitario \
        WHERE SC.cod_ibge IN({sectors});"
        
    try:
        cur.execute(query)
        result = cur.fetchall()
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

---

In [ ]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [ ]:
data = pd.read_csv('../files/inputs/data.csv')

In [ ]:
for index, row in data.iterrows():
    filename = str(int(row['filename']))
    coords = f"{row['lat']},{row['lng']}"
    polygon = get_ia(coords)
    polygon = polygon['geometry'].to_wkt()[0]

    conn, cur = open_connection()
    sectors = get_sectors(conn, cur, polygon)
    ia_data = get_data(conn, cur, sectors)
    cols = [desc[0].upper() for desc in cur.description]
    close_connection(conn, cur)

    df = pd.DataFrame(data = ia_data, columns = cols)
    df.to_csv(f"../files/outputs/{filename}.csv", index = False)
    print(f"File {filename}.csv created with {len(df)} sectors.")